In [1]:
import numpy as np
import cv2

In [2]:
image = cv2.imread('sun1.jpg')
ori = image.copy()

r = 800.0 / image.shape[0]
dim = (int(image.shape[1] * r), 800)
image = cv2.resize(image, dim, interpolation = cv2.INTER_AREA)

gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
blur = cv2.GaussianBlur(gray, (17, 17), 0)
#blur = cv2.bilateralFilter(image,5,50,50)
#blur = cv2.GaussianBlur(gray, (5, 5), 0)
#blur = cv2.medianBlur(image,5)
#_, thresh = cv2.threshold(blur, 127, 255, cv2.THRESH_BINARY)
edge = cv2.Canny(blur, 25, 159)
#edge = cv2.medianBlur(edge,3)

print("step 1: edge detection")

cv2.namedWindow('blur', cv2.WINDOW_NORMAL)
cv2.namedWindow('image', cv2.WINDOW_NORMAL)
cv2.namedWindow('edge', cv2.WINDOW_NORMAL)
#cv2.namedWindow('thresh', cv2.WINDOW_NORMAL)
cv2.imshow('blur', blur)
cv2.imshow('image', image)
#cv2.imshow('thresh', thresh)
cv2.imshow('edge', edge)

cv2.waitKey(0)
cv2.destroyAllWindows()

step 1: edge detection


In [3]:
cnts, _ = cv2.findContours(edge.copy(), cv2.RETR_LIST, cv2.CHAIN_APPROX_SIMPLE)

cnts = sorted(cnts, key = cv2.contourArea, reverse = True)[:5]

for c in cnts:
    peri = cv2.arcLength(c, True)
    approx = cv2.approxPolyDP(c, 0.02 * peri, True)
    
    if len(approx) == 4:
        screenCnt = approx
        break

print("step 2: find contours")

cv2.drawContours(image, [screenCnt], -1, (0, 255, 0), 2)
cv2.imshow('outline', image)

cv2.waitKey(0)
cv2.destroyAllWindows()

step 2: find contours


In [4]:
def order_points(pts):
    rect = np.zeros((4, 2), dtype='float32')
    
    s = pts.sum(axis = 1)
    
    rect[0] = pts[np.argmin(s)]
    rect[2] = pts[np.argmax(s)]
    
    diff = np.diff(pts, axis=1)
    rect[1] = pts[np.argmin(diff)]
    rect[3] = pts[np.argmax(diff)]
    
    return rect

In [6]:
rect = order_points(screenCnt.reshape(4,2) / r)
(topLeft, topRight, bottomRight, bottomLeft) = rect

w1 = abs(bottomRight[0] - bottomLeft[0])
w2 = abs(topRight[0] - topLeft[0])
h1 = abs(topRight[1] - bottomRight[1])
h2 = abs(topLeft[1] - bottomLeft[1])
maxWidth = max([w1, w2])
maxHeight = max([h1, h2])

dst = np.float32([[0, 0], [maxWidth - 1, 0], [maxWidth-1, maxHeight-1], [0, maxHeight - 1]])

M = cv2.getPerspectiveTransform(rect, dst)

warped = cv2.warpPerspective(ori, M, (maxWidth, maxHeight))

print('step 3: apply perspective transform')
cv2.imshow('warped', warped)

cv2.waitKey(0)
cv2.destroyAllWindows()

<ipython-input-6-1dbf53942fbf>:15: DeprecationWarning: an integer is required (got type numpy.float32).  Implicit conversion to integers using __int__ is deprecated, and may be removed in a future version of Python.
  warped = cv2.warpPerspective(ori, M, (maxWidth, maxHeight))


step 3: apply perspective transform
